In [16]:
import pandas as pd 
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

# Q1 Downloading the data

Downloading from https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page, we want to 
download the yellow taxi data for Jan, Feb 2023

In [3]:
from utils import download_data_files

download_data_files(
    "yellow_tripdata_2023-01.parquet", # jan 2023
    "yellow_tripdata_2023-02.parquet", # feb 2023
)

Downloaded yellow_tripdata_2023-01.parquet with 3066766 rows and 19 columns
Downloaded yellow_tripdata_2023-02.parquet with 2913955 rows and 19 columns


In [4]:
from utils import load_file 
df_jan = load_file("yellow_tripdata_2023-01.parquet")
print(f"Columns {df_jan.shape[1]}")


Columns 19


# Q2: Computing Duration

What's the standard deviation of the trips duration in January

In [5]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [6]:
from utils import clean_and_calculate_fields
df_jan_cleaned = clean_and_calculate_fields(df_jan)
df_jan_cleaned.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [7]:
df_jan_cleaned[['tpep_dropoff_datetime','tpep_pickup_datetime','duration']]

,tpep_dropoff_datetime,tpep_pickup_datetime,duration
0,2023-01-01 00:40:36,2023-01-01 00:32:10,8.433333
1,2023-01-01 01:01:27,2023-01-01 00:55:08,6.316667
2,2023-01-01 00:37:49,2023-01-01 00:25:04,12.750000
3,2023-01-01 00:13:25,2023-01-01 00:03:48,9.616667
4,2023-01-01 00:21:19,2023-01-01 00:10:29,10.833333
...,...,...,...
3066761,2023-02-01 00:12:33,2023-01-31 23:58:34,13.983333
3066762,2023-01-31 23:50:36,2023-01-31 23:31:09,19.450000
3066763,2023-01-31 23:25:36,2023-01-31 23:01:05,24.516667
3066764,2023-01-31 23:53:00,2023-01-31 23:40:00,13.000000


In [8]:
std_dev = df_jan_cleaned.std(numeric_only=True)
print("Std Dev for duration", std_dev['duration'])


Std Dev for duration 42.59435124195458


# Q3 Dropping Outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?


In [9]:
orig_records = df_jan_cleaned.shape[0]
print("Original Record count", orig_records)

df_jan_no_outliers = df_jan_cleaned[(df_jan_cleaned.duration >= 1) & (df_jan_cleaned.duration <= 60)]
filtered_count = df_jan_no_outliers.shape[0]
print("Filtered count", filtered_count)
print(f"Kept {filtered_count/orig_records:2.2%}")

Original Record count 3066766
Filtered count 3009173
Kept 98.12%


# Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [22]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
train_dicts = df_jan_no_outliers[categorical + numerical].to_dict(orient='records')


dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train
print("Columns", X_train.shape[1])

Columns 516


# Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

 - Train a plain linear regression model with default parameters, where duration is the response variable
 - Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [23]:
target = 'duration'
y_train = df_jan_no_outliers[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

7.658404649954137

# Q6. Evaluating the Model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [39]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [40]:
df_val= read_dataframe('./data/yellow_tripdata_2023-02.parquet')

In [37]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
target = 'duration'
y_val = df_val[target].values

y_pred = lr.predict(X_val)
root_mean_squared_error(y_val, y_pred)

43.2438310689848